In [1]:
from collections import defaultdict, Counter
import numpy as np

# Example train and test corpus (list of sentences)
train_corpus = [
    "a quick brown fox jumps over the lazy dog",
    "the fox is quick and brown",
    "quick brown fox",
]
test_corpus = [
    "a quick fox jumps",
    "the dog is lazy",
]


In [2]:
def get_ngrams(sentence, n):
    words = ['<s>'] * (n - 1) + sentence.split() + ['</s>']
    return [tuple(words[i:i+n]) for i in range(len(words) - n + 1)]

def count_ngrams(corpus, n):
    ngram_counts = Counter()
    for sent in corpus:
        ngram_counts.update(get_ngrams(sent, n))
    return ngram_counts


In [3]:
class LaplaceNGramLM:
    def __init__(self, corpus, n):
        self.n = n
        self.ngram_counts = count_ngrams(corpus, n)
        self.nm1gram_counts = count_ngrams(corpus, n-1) if n > 1 else None
        self.vocab = set(word for sent in corpus for word in sent.split())
        self.vocab_size = len(self.vocab)
    def prob(self, ngram):
        if self.n == 1:
            c = self.ngram_counts[ngram]
            return (c + 1) / (sum(self.ngram_counts.values()) + self.vocab_size)
        else:
            c = self.ngram_counts[ngram]
            d = self.nm1gram_counts[ngram[:-1]]
            return (c + 1) / (d + self.vocab_size)
    def sent_prob(self, sentence):
        ngrams = get_ngrams(sentence, self.n)
        return np.product([self.prob(ngram) for ngram in ngrams])


In [4]:
class InterpolatedNGramLM:
    def __init__(self, corpus, n, lambdas=None):
        self.n = n
        self.ngram_counts = [count_ngrams(corpus, i+1) for i in range(n)]
        self.vocab = set(word for sent in corpus for word in sent.split())
        self.vocab_size = len(self.vocab)
        self.lambdas = lambdas or [1/n] * n
    def prob(self, ngram):
        probs = []
        for i in range(self.n):
            gram = ngram[-(i+1):]
            if i == 0:
                prob = (self.ngram_counts[0][(gram[0],)] + 1) / (sum(self.ngram_counts[0].values()) + self.vocab_size)
            else:
                d = self.ngram_counts[i-1][gram[:-1]]
                prob = (self.ngram_counts[i][gram] + 1) / (d + self.vocab_size)
            probs.append(self.lambdas[i] * prob)
        return sum(probs)
    def sent_prob(self, sentence):
        ngrams = get_ngrams(sentence, self.n)
        return np.product([self.prob(ngram) for ngram in ngrams])


In [5]:
class InterpolatedNGramLM:
    def __init__(self, corpus, n, lambdas=None):
        self.n = n
        self.ngram_counts = [count_ngrams(corpus, i+1) for i in range(n)]
        self.vocab = set(word for sent in corpus for word in sent.split())
        self.vocab_size = len(self.vocab)
        self.lambdas = lambdas or [1/n] * n
    def prob(self, ngram):
        probs = []
        for i in range(self.n):
            gram = ngram[-(i+1):]
            if i == 0:
                prob = (self.ngram_counts[0][(gram[0],)] + 1) / (sum(self.ngram_counts[0].values()) + self.vocab_size)
            else:
                d = self.ngram_counts[i-1][gram[:-1]]
                prob = (self.ngram_counts[i][gram] + 1) / (d + self.vocab_size)
            probs.append(self.lambdas[i] * prob)
        return sum(probs)
    def sent_prob(self, sentence):
        ngrams = get_ngrams(sentence, self.n)
        return np.product([self.prob(ngram) for ngram in ngrams])


In [6]:
def perplexity(lm, corpus):
    N = 0
    log_prob = 0
    for sent in corpus:
        ngrams = get_ngrams(sent, lm.n)
        for ng in ngrams:
            prob = lm.prob(ng)
            if prob == 0: # Avoid log(0)
                prob = 1e-10
            log_prob += -np.log(prob)
            N += 1
    return np.exp(log_prob / N)

unigram_laplace = LaplaceNGramLM(train_corpus, 1)
bigram_laplace = LaplaceNGramLM(train_corpus, 2)
trigram_laplace = LaplaceNGramLM(train_corpus, 3)

print("Bigram Laplace Perplexity:", perplexity(bigram_laplace, test_corpus))
print("Trigram Interpolation Perplexity:",
      perplexity(InterpolatedNGramLM(train_corpus, 3, [0.2, 0.3, 0.5]), test_corpus))


Bigram Laplace Perplexity: 9.291313290738232
Trigram Interpolation Perplexity: 9.485268178704718


In [7]:
def predict_next_word(lm, prefix):
    prefix = prefix.split()
    if len(prefix) < lm.n - 1:
        prefix = ['<s>'] * (lm.n - 1 - len(prefix)) + prefix
    else:
        prefix = prefix[-(lm.n - 1):]
    candidates = list(lm.vocab)
    scores = []
    for cand in candidates:
        ngram = tuple(prefix + [cand])
        scores.append((cand, lm.prob(ngram)))
    scores.sort(key=lambda x: -x[1])
    return scores[:3]

# Example for 'a quick'
print("Predictions:", predict_next_word(bigram_laplace, 'a quick'))


Predictions: [('brown', 0.21428571428571427), ('and', 0.14285714285714285), ('quick', 0.07142857142857142)]


In [8]:
class StupidBackoffNGramLM:
    def __init__(self, corpus, n, alpha=0.4):
        self.n = n
        self.alpha = alpha
        self.counts = [count_ngrams(corpus, i+1) for i in range(n)]
        self.vocab = set(word for sent in corpus for word in sent.split())
        self.total_unigrams = sum(self.counts[0].values())

    def score(self, ngram):
        n = len(ngram)
        if n == 1:
            return self.counts[0][ngram] / self.total_unigrams
        elif self.counts[n-1][ngram] > 0:
            prefix = ngram[:-1]
            return self.counts[n-1][ngram] / self.counts[n-2][prefix]
        else:
            # Recursively back off to lower order, multiplying by alpha
            return self.alpha * self.score(ngram[1:])

    def sent_score(self, sentence):
        ngrams = get_ngrams(sentence, self.n)
        scores = [self.score(ngram) for ngram in ngrams]
        # For product of scores; to avoid underflow, work in log space if needed
        return np.product(scores)
